In [22]:
!pip install selenium webdriver-manager pandas


In [24]:
import re
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager


In [25]:
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920x1080")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)


In [26]:
driver.get("https://www.rohlik.cz/hledat?q=ml%C3%A9%C4%8Dn%C4%9B%20v%C3%BDrobky&companyId=1")
time.sleep(5)  # počkej na načtení stránky


In [27]:
data = []

# Názvy a ceny (celkem)
product_names  = driver.find_elements(By.CSS_SELECTOR, "[data-test='productCard-body-name']")
product_prices = driver.find_elements(By.CSS_SELECTOR, "[data-test='productCard-body-price']")

# Hmotnost a cena/kg (unit price)
product_weights    = driver.find_elements(By.CSS_SELECTOR, "[data-test='productCard-footer-amount']")
product_unitprices = driver.find_elements(By.CSS_SELECTOR, "[data-test='productCard-footer-unitPrice']")

# Odkaz na detail produktu
product_links = driver.find_elements(By.CSS_SELECTOR, "[data-test='productCard-header-image'] a")

# Ověříme, zda je všech pět seznamů stejné délky
if (
    len(product_names)     == len(product_prices)   ==
    len(product_weights)   == len(product_unitprices) ==
    len(product_links)
):
    for name_el, price_el, weight_el, unitprice_el, link_el in zip(
        product_names, product_prices, product_weights, product_unitprices, product_links
    ):
        # Název
        name_text = name_el.text.strip()

        # Cena celkem
        raw_price = price_el.text.replace("\n", "").replace("\u00a0", "").strip()
        raw_price = raw_price.replace("Kč", "").strip()
        price_with_dot = re.sub(r'\s+', '.', raw_price)
        if re.match(r'^\d{4}$', price_with_dot):
            price_with_dot = price_with_dot[:2] + '.' + price_with_dot[2:]
        elif re.match(r'^\d{3}$', price_with_dot):
            price_with_dot = price_with_dot[0] + '.' + price_with_dot[1:]
        try:
            price_float = float(price_with_dot)
            final_price = f"{price_float:.2f}".replace('.', ',') + " Kč"
        except ValueError:
            final_price = "0,00 Kč"

        # Hmotnost
        weight_text = weight_el.text.strip()

        # Cena za 1 kg
        unit_text = unitprice_el.text.strip()

        # Odkaz na detail
        relative_url = link_el.get_attribute("href")
        full_url = relative_url

        data.append([name_text, final_price, weight_text, unit_text, full_url])
else:
    print("⚠️ POZOR! Počet prvků nesedí. Zkontroluj selektory.")

# Zavření driveru
driver.quit()

print(f"Načteno produktů: {len(data)}")


Načteno produktů: 12


In [28]:
df = pd.DataFrame(data, columns=["Název", "Cena", "Hmotnost", "Cena/kg", "Odkaz"])

# Náhled dat v notebooku
df.head(10)


,Název,Cena,Hmotnost,Cena/kg,Odkaz
0,Philadelphia Original Smetanový sýr,"68,00 Kč",200 g,340 Kč/kg,https://www.rohlik.cz/1410229-philadelphia-ori...
1,Philadelphia Original Smetanový sýr,"61,90 Kč",125 g,495.2 Kč/kg,https://www.rohlik.cz/761943-philadelphia-orig...
2,Président Delikátní tvaroh pažitka,"27,90 Kč",125 g,223.2 Kč/kg,https://www.rohlik.cz/1437935-president-delika...
3,Président Delikátní tvaroh ředkvička,"27,90 Kč",125 g,223.2 Kč/kg,https://www.rohlik.cz/1437934-president-delika...
4,Président Delikátní tvaroh rajče,"27,90 Kč",125 g,223.2 Kč/kg,https://www.rohlik.cz/1437936-president-delika...
5,Président Delikátní tvaroh,"27,90 Kč",125 g,223.2 Kč/kg,https://www.rohlik.cz/1437933-president-delika...
6,Eru Gouda tavený sýr,"36,90 Kč",100 g,369 Kč/kg,https://www.rohlik.cz/1437187-eru-gouda-taveny...
7,Pragolaktos Trvanlivé plnotučné mléko bez lakt...,"30,90 Kč",1 l,30.9 Kč/l,https://www.rohlik.cz/1422152-pragolaktos-trva...
8,Parmareggio Tavený sýr s Parmigiano Reggiano,"55,40 Kč",140 g,395.71 Kč/kg,https://www.rohlik.cz/1376087-parmareggio-tave...
9,"Pragolaktos Trvanlivé mléko 3,5% plnotučné","28,90 Kč",1 l,28.9 Kč/l,https://www.rohlik.cz/1420012-pragolaktos-trva...


In [29]:
df.to_csv("rohlik.csv", index=False, encoding="utf-8")
print("✅ Data byla uložena do souboru rohlik.csv")


✅ Data byla uložena do souboru rohlik.csv
